In [1]:
import pandas as pd
import dask as dd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.formula.api as smf
import statistics

from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats

import requests
import yfinance as yf


/usr/local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
sns.set()

In [3]:
df = pd.read_csv('../data.csv')

/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_1179/3454644713.py:1: DtypeWarning: Columns (3,4,7,8,11,12,14,15,16,19,45,46,49,56,69,83,95,98,99,100,104,108,109,119,120,121,122,123,124,125,127,128,129,141,143,144,145,146,147,148,155,188,217,218,219,220,221,222,223,224,226,229,232,233,234,235,236,237,238,239,240,243,244,245,246,248,249,250,252) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data.csv')


In [4]:
df = df[df['psic'].notna()] #drop 45 NaN participant SIC
telDF = df[df['psicp'].str.contains('481')] #Create df who have at least one participant in the tellcom industry with SIC: https://siccode.com/sic-code/481/telephone-communications
telDF = telDF.drop('Unnamed: 0', axis=1)
telDF = telDF.reset_index(drop=True)

#Export
telDF.to_csv('teldf.csv', index=False) #create csv sample

In [7]:
telDF['equity_stake'].value_counts()

N    8076
Y       9
Name: equity_stake, dtype: int64

### Analyse companies in DF

In [ ]:
kortDF = telDF[['da', 'p', 'psicp', 'nump', 'pticker', 'sfc', 'jvf', 'equity_stake']]

kortDF['nump'] = kortDF['nump'].astype(int)

pattern = r'\\n' #Regex pattern to split name

#Create seperate columns for each participsnt
for n in range(1, (int(kortDF["nump"].max()) + 1)): 
	kortDF['p' + str(n) + "name"] = kortDF['p'].str.split(pattern, expand=True)[n-1] #Create seperate columns for each possible participant

	kortDF["p" + str(n) + "sicp"] = kortDF['psicp'].str.split(pattern, expand=True)[n-1] #Create seperate columns to split participant SIC codes

	kortDF["p" + str(n) + "ticker"] = kortDF['pticker'].str.split(pattern, expand=True)[n-1]#Create seperate collumns for participants identifiers

In [16]:
kortDF['equity_stake'].value_counts()

N    8076
Y       9
Name: equity_stake, dtype: int64

### Engineer diversity

In [ ]:
kortDF['diversity'] = 0

kortDF.loc[(kortDF.equity_stake == 'Y') & (kortDF.jvf == "No"), 'diversity'] = 1
kortDF.loc[(kortDF.equity_stake == 'Y') & (kortDF.jvf == "Yes"), 'diversity'] = 2



In [20]:
kortDF.diversity.value_counts()

0    8076
2       5
1       4
Name: diversity, dtype: int64

In [8]:
#Pattern to select names of individual companies
pattern = r'p\d+ticker'

#Get collumns of indivisual companies
foo = kortDF.filter(regex=pattern, axis=1)

#Create DF of companies with their occurances in the entire dataset
companies = pd.DataFrame(foo.apply(lambda x: x.value_counts())
					.sum(axis=1, skipna=True)
					.sort_values(ascending= False), columns=['alliance_count']).reset_index()

companies = companies.rename(columns={'index': 'ticker'})

#slice and keep only 100 top companies by alliance count
companies = companies.nlargest(102, columns='alliance_count')
companies = companies[companies.ticker != ''].reset_index(drop=True)

In [ ]:
companies['name'] = ''

#Get names for companies who are publically listed
def getName(ticker):
	print(ticker)
	response = yf.Ticker(ticker)

	if (response and ('longName' in response.info)):
		return response.info['longName']
	else:
		return None


def apply_complex_function(x): return getName(x['ticker'])

companies['name'] = companies.apply(apply_complex_function, axis=1)

In [135]:
companies.to_csv('companies.csv')

In [11]:
companies = pd.read_csv('companies.csv')

In [12]:
import urllib3

fin_data = {}

#Functioon to source historic price data through yahoo Finance
def get_yfinance_data(ticker):
	print(ticker)
	
	url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1=1072915200&period2=1419984000&interval=1mo&events=history&includeAdjustedClose=true'

	headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

	if (requests.get(url, headers=headers).status_code == 200):
		data = pd.read_csv(url)
	else:
		data = None
	
	return data

#Funciton to split and arragne data to before and after discontinuity for a given company
def getPrices(ticker):
	te = get_yfinance_data(ticker)

	if (type(te) != pd.core.frame.DataFrame):
		return (None, None)

	te['Date'] = pd.to_datetime(te['Date'])
	te = te.set_index('Date', drop=True)
	p_before = (te[te.index.year <= 2009]['Close']).mean()
	p_after = (te[te.index.year > 2009]['Close']).mean()

	return (p_before, p_after)


price_bd = []
price_ad = []

#Create two sample lists for the prices averege price before and after
for t in companies['ticker']:
	vals = getPrices(t)

	price_bd.append(vals[0])
	price_ad.append(vals[1])

T
ERIC
BTE
LU
MCIC
9432
MSFT
9437
MOT
BEL
FON
BLS
9984
TLS
VOD
TEF
IBM
NOK1V
9431
Z74
GTE
9433
NYN
BT
OLIV
DTE
SKM
6701
AIT
SBC
NTL
CSCO
UYW
TM
6702
0013
KTC
CHU
INTC
CWO
6752
PHI
AV
KPN
HWP
9434
CW
SIE
QCOM
WCOM
TI
6501
FTEL
6758
STEI
Q
TEL
NT
GLO
CGE.PA
8053
MTEL
HKT
AT
2412
TELN
TLKM
500483
CW.
005930
nan
LVLT
MCAWA
VZ
SUNW
CHL
AWE
8058
AOL
YHOO
ISAT
8031
GBLX
ATI
BCE
6503
032640
MTOH
6502
TELMEX
PAC
ETEL
8002
8001
NOVL
MBT
ASY
TLD
NWS
QTEL
CHA


In [13]:
#Add price lists to dataframe
companies['price_bd'] = price_bd
companies['price_ad'] = price_ad

In [15]:
companies.to_csv('c_data.csv')

In [27]:
companies = pd.read_csv('./c_data.csv')
companies.drop("Unnamed: 0.1", axis=1, inplace=True)
companies.drop("Unnamed: 0", axis=1, inplace=True)

In [32]:
companies.ticker

0         T
1      ERIC
2       BTE
3        LU
4      MCIC
       ... 
96      ASY
97      TLD
98      NWS
99     QTEL
100     CHA
Name: ticker, Length: 101, dtype: object

In [43]:
pattern = r'pticker|diversity'

wall = kortDF.filter(regex = pattern)

companies['diversity'] = 0
companies['diversity_var'] = 0

In [48]:
companies['diversity'] = companies['ticker'].apply(lambda x: len(wall.loc[(wall.pticker.str.contains(f'{x}'))]['diversity'].value_counts()))


In [51]:
companies['diversity_var'] = companies['ticker'].apply(lambda x: (wall.loc[(wall.pticker.str.contains(f'{x}'))]['diversity']).var())

In [52]:
companies

,ticker,alliance_count,name,price_bd,price_ad,diversity,diversity_var
0,T,411.0,AT&T Inc.,22.367615,24.207578,3,0.004833
1,ERIC,185.0,Telefonaktiebolaget LM Ericsson (publ),14.161389,11.360500,1,0.000000
2,BTE,134.0,NaN,NaN,NaN,1,0.000000
3,LU,124.0,Lufax Holding Ltd,NaN,NaN,1,0.000000
4,MCIC,116.0,"MultiCorp International, Inc.",10.848227,0.006962,1,0.000000
...,...,...,...,...,...,...,...
96,ASY,16.0,NaN,18.754756,0.045172,1,0.000000
97,TLD,15.0,NaN,NaN,NaN,1,0.000000
98,NWS,15.0,News Corporation,NaN,16.591667,1,0.000000
99,QTEL,15.0,NaN,NaN,NaN,1,0.000000


In [54]:
companies.to_csv('c_data_var.csv')